In [2]:
from __future__ import division
import os, sys
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches

import pandas as pd
from svgpathtools import parse_path, wsvg, svg2paths

/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/spatial/__init__.py:96: ImportWarning: Not importing directory '/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/spatial/qhull': missing __init__.py
  from .qhull import *
/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/optimize/_minimize.py:37: ImportWarning: Not importing directory '/Users/megumisano/anaconda/envs/py27/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb


### setup

In [3]:
# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis','python') not in sys.path:
    sys.path.insert(1,os.path.join(proj_dir,'analysis','python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

import svg_rendering_helpers as srh
reload(srh)

<module 'svg_rendering_helpers' from '/Users/megumisano/graphical_conventions/analysis/python/svg_rendering_helpers.pyc'>

In [4]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['3dObjects']
coll = db['graphical_conventions']

# which iteration name should we use?
## "good" iterations that we currently combine: ['run3_size4_waiting', 'run4_generalization']
iterationNames = ['run3_size4_waiting', 'run4_generalization']

#### wsvg arguments
```
wsvg(paths=None, colors=None, filename=join, stroke_widths=None, nodes=None, node_colors=None, node_radii=None, openinbrowser=False, timestamp=False, margin_size=0.1, mindim=600, dimensions=None, viewbox=None, text=None, text_path=None, font_size=None, attributes=None, svg_attributes=None)
```

### what we need (tests on single sketches)

- render_svg
- svg_to_png
- svg2paths (already defined in svgpathtools)

In [5]:
### rendering stroke-by-stroke 
stroke_dir = os.path.join(sketch_dir, 'sample_subsets')

In [103]:
reload(srh)
## render out svg
srh.render_svg(parsed,base_dir=stroke_dir)

In [7]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(stroke_dir,'svg'))

In [8]:
## convert to png
srh.svg_to_png(svg_paths,base_dir=stroke_dir)

convert /Users/megumisano/graphical_conventions/sketches/stroke_analysis/svg/4479-3f10bccf-ef14-4c6a-a2f5-333f692f78d8_6_waiting_01_0_run4_generalization_0_end.svg /Users/megumisano/graphical_conventions/sketches/stroke_analysis/png/4479-3f10bccf-ef14-4c6a-a2f5-333f692f78d8_6_waiting_01_0_run4_generalization_0_end.png


### svg rendering with stroke deletion in both directions

In [8]:
D = pd.read_csv(os.path.join(results_dir, 'graphical_conventions.csv'))

In [6]:
reload(srh)
for g in D['gameID].unique()':
    D_game = D[D['gameID'] == g]
    print "rendering sketches from game " + g + " with " + str(len(D_game)) + " trials"
    for t in range(1, 41):
        D_trial = D_game[D_game['trialNum'] == t]
        iterationName = 'run4_generalization' if list(D_trial['Generalization'])[0] == 'between' else 'run3_size4_waiting'
        trial_level_path = g + '_' + str(t) + '_' + list(D_trial['target'])[0] + '_' + str(list(D_trial['repetition'])[0]) + '_' + iterationName
        if int(trial_level_path.split('_')[1]) > 40:
            print(trial_level_path)
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['trialNum'] == t)]['svgString'].unique()[0])
        assert len(svg_list) != 0
        # deleting from the end 
        for num_strokes in range(len(svg_list)): # 0 to 7
            stroke_and_direction = '_' + str(num_strokes) + '_start.svg'  # deleting how many strokes? 
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(num_strokes,len(svg_list))]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)
        # and deleting from the start 
        for num_strokes in range(len(svg_list)): # 0 to 7  
            stroke_and_direction = '_' + str(num_strokes) + '_end.svg' # deleting how many strokes? 
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(0, len(svg_list) - num_strokes)]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)

SyntaxError: invalid syntax (<ipython-input-6-72be75cb7584>, line 3)

In [11]:
import ast

In [20]:
# self-similarity rendering 
reload(srh)
for g in D['gameID'].unique():
    D_game = D[D['gameID'] == g]
    print("rendering sketches from game " + g + " with " + str(len(D_game)) + " trials")
    for t in range(1, 41):
        D_trial = D_game[D_game['trialNum'] == t]
        iterationName = 'run4_generalization' if list(D_trial['generalization'])[0] == 'between' else 'run3_size4_waiting'
        trial_level_path = g + '_' + str(t) + '_' + list(D_trial['target'])[0] + '_' + str(list(D_trial['repetition'])[0]) + '_' + iterationName
        if int(trial_level_path.split('_')[1]) > 40:
            print(trial_level_path)
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['trialNum'] == t)]['svgString'].unique()[0])
        assert len(svg_list) != 0
        for num_strokes in range(len(svg_list)): # 0 to 7 - 8 strokes
            stroke_and_direction = '_' + str(num_strokes) + '.svg'  # kth-index stroke deleted
            stroke_level_path = trial_level_path + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(len(svg_list)) if i != num_strokes]
            ## render out svg
            if len(parsed) > 0:
                srh.render_svg(parsed, base_dir=stroke_dir, out_fname=stroke_level_path)
#         # and deleting from the start 
#         for num_strokes in range(len(svg_list)): # 0 to 7  
#             stroke_and_direction = '_' + str(num_strokes) + '_end.svg' # deleting how many strokes? 
#             stroke_level_path = trial_level_path + stroke_and_direction
#             parsed = [parse_path(svg_list[i]) for i in range(0, len(svg_list) - num_strokes)]
#             ## render out svg
#             srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)

rendering sketches from game 0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5 with 40 trials
rendering sketches from game 2040-7604415a-2963-4416-a78c-e2fdc7e670ac with 40 trials
rendering sketches from game 8384-deeaec6b-d2a1-4f6e-ae6c-6140dfbb4071 with 40 trials
rendering sketches from game 6388-4e8fd744-2165-44fc-866a-5d68aff23732 with 40 trials
rendering sketches from game 1979-cf0efe08-9b6f-49a7-8200-43f0e7744dbc with 40 trials
rendering sketches from game 2349-8db5cdc6-e266-4486-a9c2-b99b80593419 with 40 trials
rendering sketches from game 9614-d370a26e-7936-49e5-add7-05e6d31dda4a with 40 trials
rendering sketches from game 6459-b31ded08-1443-4005-8cea-256cf02be3df with 40 trials
rendering sketches from game 0286-a442ec93-ad78-42fa-b3e9-083da9a64c4d with 40 trials
rendering sketches from game 4152-970406c3-17ba-435b-a56b-4f1f79f00119 with 40 trials
rendering sketches from game 4529-638b7428-2c20-4fb2-a388-1c7513f675d1 with 40 trials
rendering sketches from game 3522-5f73221e-1d4d-4e32-8

In [ ]:
# self-similarity GREEDY LESIONING ALGORITHM using binary lesion representations 
import ast
from itertools import combinations
random.seed(42)
reload(srh)
for g in D['gameID'].unique():
    D_game = D[D['gameID'] == g]
    print("rendering sketches from game " + g + " with " + str(len(D_game)) + " trials")
    for t in range(1, 41):
        D_trial = D_game[D_game['trialNum'] == t]
        iterationName = 'run4_generalization' if list(D_trial['generalization'])[0] == 'between' else 'run3_size4_waiting'
        trial_level_path = g + '_' + str(t) + '_' + list(D_trial['target'])[0] + '_' + str(list(D_trial['repetition'])[0]) + '_' + iterationName
        if int(trial_level_path.split('_')[1]) > 40:
            print(trial_level_path)
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['trialNum'] == t)]['svgString'].unique()[0])
        assert len(svg_list) != 0
        total_num_strokes = len(svg_list)
        for num_stroke in range(total_num_strokes): # 0 to 7 - 8 strokes
            comb_list = list(combinations(range(total_num_strokes), num_stroke))
            len(comb_list) == 2**num_stroke
            for comb_index, comb in enumerate(comb_list): 
                stroke_and_direction = '_' + str(num_stroke) + '_' + str(comb_index) + '.svg'  # kth-index stroke deleted
                stroke_level_path = trial_level_path + stroke_and_direction
                parsed = [parse_path(svg_list[i]) for i in comb]
                ## render out svg
                if len(parsed) > 0:
                    srh.render_svg(parsed, base_dir=stroke_dir, out_fname=stroke_level_path)

rendering sketches from game 7053-d1f771d6-a55b-4681-961f-5a9ebb43cbdb with 40 trials
rendering sketches from game 3558-8fa206a2-9b8c-4d67-a4ab-0832dc0d0f25 with 40 trials
rendering sketches from game 6964-d6de9966-f3dd-45df-8d24-d5bfd5d46eb8 with 40 trials
rendering sketches from game 4853-a05bfdd7-095c-4b93-bdde-2b99fd29c7a2 with 40 trials
rendering sketches from game 5616-7114c020-0e81-482c-952b-b26fc66076a0 with 40 trials


In [168]:
import ast
from matplotlib import colors
# to_compare = [['run3_1820-86afdf51-4b4c-43a9-a9e1-e6e20618de1b_3_repeated_waiting_02_0',
#               'run3_1820-86afdf51-4b4c-43a9-a9e1-e6e20618de1b_12_repeated_waiting_02_1']]
to_compare = [['run4_0647-0ece0513-aa05-4676-bb63-3845126ce2c1_1_repeated_dining_04_0',
              'run4_0647-0ece0513-aa05-4676-bb63-3845126ce2c1_11_repeated_dining_04_1']]
#               ['run4_0836-215b3490-cdf2-4a13-b55a-ea90f7b1903b_2_repeated_dining_03_0',
#                'run4_0836-215b3490-cdf2-4a13-b55a-ea90f7b1903b_10_repeated_dining_03_1']
#              ]

In [99]:
to_compare = [['run3_0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5_15_repeated_dining_05_2', 
               'run3_0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5_20_repeated_dining_05_3'],
                ['run4_6333-9ef157b9-68ab-438e-bb27-f588abc2c906_23_repeated_dining_05_4', 
              'run4_6333-9ef157b9-68ab-438e-bb27-f588abc2c906_19_repeated_dining_05_3'],
             ['run4_0955-60582965-41c6-492d-aa67-98e15630db39_2_repeated_dining_01_0',
             'run4_0955-60582965-41c6-492d-aa67-98e15630db39_10_repeated_dining_01_1'],
             ['run3_4529-638b7428-2c20-4fb2-a388-1c7513f675d1_7_repeated_waiting_05_0',
           'run3_4529-638b7428-2c20-4fb2-a388-1c7513f675d1_9_repeated_waiting_05_1'], 
           ['run4_7940-e30c3a20-dd99-4efc-ae92-f650a7f1d6e6_20_repeated_waiting_02_3',
           'run4_7940-e30c3a20-dd99-4efc-ae92-f650a7f1d6e6_21_repeated_waiting_02_4']]

In [142]:
# customized rendering for qualitative analyses 
reload(srh)
# to_compare = [['run3_0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5_1_repeated_dining_04_0', 
#             'run3_0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5_12_repeated_dining_04_1'], 
#            ['run3_4529-638b7428-2c20-4fb2-a388-1c7513f675d1_7_repeated_waiting_05_0',
#            'run3_4529-638b7428-2c20-4fb2-a388-1c7513f675d1_9_repeated_waiting_05_1'], 
#            ['run4_7940-e30c3a20-dd99-4efc-ae92-f650a7f1d6e6_20_repeated_waiting_02_3',
#            'run4_7940-e30c3a20-dd99-4efc-ae92-f650a7f1d6e6_21_repeated_waiting_02_4']]

for pair in to_compare:
    for sketch in pair:
        print "rendering sketch: " + sketch 
        g = sketch.split('_')[1]
        t = sketch.split('_')[2]
        ## find all svg paths of each sketch 
        ## delete one and render (with path name indicating which stroke was deleted)
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['trialNum'] == int(t))]['svgString'].unique()[0])
        assert len(svg_list) != 0
        for num_strokes in range(len(svg_list)): # 0 to 7
            stroke_and_direction = '_' + str(num_strokes) + '_dummie2.svg'  # deleting which stroke? 
            stroke_level_path = sketch + stroke_and_direction
            parsed = [parse_path(svg_list[i]) for i in range(len(svg_list)) if i != num_strokes]
            ## render out svg
            srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)

rendering sketch: run3_1820-86afdf51-4b4c-43a9-a9e1-e6e20618de1b_3_repeated_waiting_02_0
rendering sketch: run3_1820-86afdf51-4b4c-43a9-a9e1-e6e20618de1b_12_repeated_waiting_02_1


In [ ]:
# self-similarity GREEDY LESIONING ALGORITHM using permutations 
import ast
from itertools import permutations
random.seed(42)
reload(srh)
for g in D['gameID'].unique():
    D_game = D[D['gameID'] == g]
    print("rendering sketches from game " + g + " with " + str(len(D_game)) + " trials")
    for t in range(1, 41):
        D_trial = D_game[D_game['trialNum'] == t]
        iterationName = 'run4_generalization' if list(D_trial['generalization'])[0] == 'between' else 'run3_size4_waiting'
        trial_level_path = g + '_' + str(t) + '_' + list(D_trial['target'])[0] + '_' + str(list(D_trial['repetition'])[0]) + '_' + iterationName
        if int(trial_level_path.split('_')[1]) > 40:
            print(trial_level_path)
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['trialNum'] == t)]['svgString'].unique()[0])
        assert len(svg_list) != 0
        total_num_strokes = len(svg_list)
        perm_list = list(set(permutations(range(total_num_strokes))))
        if total_num_strokes <= 4:
            how_many = len(perm_list)
        else:
            how_many = 100
        for perm_index, perm in enumerate(perm_list[:how_many]):
            if perm == range(total_num_strokes):
                continue
            for num_strokes, stroke_index in enumerate(perm): # 0 to 7 - 8 strokes
                stroke_and_direction = '_' + str(num_strokes) + '_' + str(perm_index) + '.svg'  # kth-index stroke deleted
                stroke_level_path = trial_level_path + stroke_and_direction
                parsed = [parse_path(svg_list[i]) for i in range(len(svg_list)) if i != num_strokes]
                ## render out svg
                if len(parsed) > 0:
                    srh.render_svg(parsed, base_dir=stroke_dir, out_fname=stroke_level_path)
#         # and deleting from the start 
#         for num_strokes in range(len(svg_list)): # 0 to 7  
#             stroke_and_direction = '_' + str(num_strokes) + '_end.svg' # deleting how many strokes? 
#             stroke_level_path = trial_level_path + stroke_and_direction
#             parsed = [parse_path(svg_list[i]) for i in range(0, len(svg_list) - num_strokes)]
#             ## render out svg
#             srh.render_svg(parsed,base_dir=stroke_dir, out_fname=stroke_level_path)

In [171]:
#### now, rendering with different colors!!!! 
reload(srh)
qd = pd.read_csv('qualitative_stroke_analysis_2.csv')
for pair in to_compare:
    for sketch in pair:
        print ('game ' + g)
        g = sketch.split('_')[1]
        if g == '0724-1a59371c-6fe6-4d6d-a987-db533e75a5e5':
            target = 'dining_05'
        elif g == '4529-638b7428-2c20-4fb2-a388-1c7513f675d1':
            target = 'waiting_05'
        elif g == '7940-e30c3a20-dd99-4efc-ae92-f650a7f1d6e6':
            target = 'waiting_02'
        elif g == '6333-9ef157b9-68ab-438e-bb27-f588abc2c906':
            target = 'dining_05'
        elif g == '0647-0ece0513-aa05-4676-bb63-3845126ce2c1':#if g == '0955-60582965-41c6-492d-aa67-98e15630db39':
            target = 'dining_04'
        elif g == '1820-86afdf51-4b4c-43a9-a9e1-e6e20618de1b':
            target = 'waiting_02'
        else:
            target = 'dining_03'
        r = sketch.split('_')[6]
        qd_ = qd[(qd['gameID'] == g) & (qd['lesioned_rep'] == int(r))]
        stroke_numbers = qd_['deleted_stroke_number'].unique()
        svg_list = ast.literal_eval(D[(D['gameID'] == g) & (D['repetition'] == int(r)) & (D['target'] == target)]['svgString'].unique()[0])
        # Color normalization 
        max_similarity = np.max(np.array(qd_['similarity']))
        min_similarity = np.min(np.array(qd_['similarity']))
        norm = colors.Normalize(vmin=min_similarity, vmax=max_similarity)
        cmap = plt.cm.get_cmap('viridis')
        stroke_colors = []
        for num_strokes in range(len(svg_list)):
            similarity = qd_[qd_['deleted_stroke_number'] == num_strokes]['similarity'].unique()[0]
            rgba = cmap(1 - norm(similarity))
            color=colors.to_hex(rgba)
            stroke_colors.append(color)
        stroke_and_direction = '_color3.svg'  # deleting which stroke? 
        stroke_level_path = sketch + stroke_and_direction
        parsed = [parse_path(svg_list[i]) for i in range(len(svg_list))]
        srh.render_svg_color(parsed, stroke_colors, base_dir=stroke_dir, out_fname=stroke_level_path)

game 0647-0ece0513-aa05-4676-bb63-3845126ce2c1
game 0647-0ece0513-aa05-4676-bb63-3845126ce2c1


In [15]:
## get svg path list
svg_paths = srh.generate_svg_path_list(os.path.join(stroke_dir,'svg'))
#svg_paths = [path for path in svg_paths if path.split('.')[0].split('/')[-1].split('_')[-1] == 'color3'] #and path.split('.')[0].split('/')[-1].split('_')[1] == '1820-86afdf51-4b4c-43a9-a9e1-e6e20618de1b'] 

In [16]:
svg_paths[0]

'/Users/megumisano/graphical_conventions/sketches/self_similarity/svg/9075-d6c9b0a6-bfb2-4148-ae19-d0da518aef97_25_waiting_04_5_run3_size4_waiting_1.svg'

In [23]:
## convert to png
reload(srh)
diffs = ['/Users/megumisano/graphical_conventions/sketches/self_similarity/svg/' + path + '.svg' for path in diff]
srh.svg_to_png(diffs,base_dir=stroke_dir)

convert /Users/megumisano/graphical_conventions/sketches/self_similarity/svg/2391-14c2ae6d-73d2-4e2a-b4e8-d9cdedbd393c_17_waiting_05_3_run4_generalization_0.svg /Users/megumisano/graphical_conventions/sketches/self_similarity/png/2391-14c2ae6d-73d2-4e2a-b4e8-d9cdedbd393c_17_waiting_05_3_run4_generalization_0.png


In [18]:
len(set(os.listdir(os.path.join(stroke_dir, 'svg'))))

14622

In [19]:
len(set(os.listdir(os.path.join(stroke_dir, 'png'))))

14621

In [20]:
svg_paths = os.listdir(os.path.join(stroke_dir, 'svg'))
png_paths = os.listdir(os.path.join(stroke_dir, 'png'))
svg_paths_ = [path.split('.')[0] for path in svg_paths]
png_paths_ = [path.split('.')[0] for path in png_paths]

In [21]:
diff = set(svg_paths_).difference(set(png_paths_))

In [22]:
len(diff)

1